In [14]:
## Installing Required Packages
import sys
import pickle
import ta
#!{sys.executable} -m pip install alpaca_trade_api
#!{sys.executable} -m pip install tqdm
#!{sys.executable} -m pip install ipywidgets
#!{sys.executable} -m pip install ta

In [15]:
## Loading Packages
import alpaca_trade_api as tradeapi
import os
import pandas
from datetime import datetime, timedelta

Project_Folder = "C://Users//" + os.getlogin() + "//documents//github//Stock-Strategy-Exploration//"

N_DAYS_AGO = 365*3
start_date = datetime.now() - timedelta(days=N_DAYS_AGO)
stock = "MSFT"

## Pulling API Keys
Keys = pandas.read_csv(os.path.join('..','Data','Keys','Live API.txt'))

## Connecting to Alpaca
api = tradeapi.REST(Keys['Key ID'][0],Keys['Secret Key'][0], api_version='v2')

## Testing API Functionality
### Get Current Account Information
account = api.get_account()
### List of Tradable Assets
assets = api.list_assets("active")
Assets = [i._raw['symbol'] for i in assets if i._raw['tradable'] & i._raw['shortable']  & i._raw['easy_to_borrow']]
### Pulling OHLC Data
bars = api.polygon.historic_agg_v2(stock, 1, 'day', _from=start_date.strftime("%Y-%m-%d"), to=datetime.now().strftime("%Y-%m-%d")).df



In [13]:
from collections import defaultdict
from tqdm.auto import tqdm
import pickle

## Amount of Historical Data to Pull
N_DAYS_AGO = 365*3

## Checking if Data Exists
if os.path.exists(Project_Folder + "Data//Historical_Data.p"):
    start_date = Historical_Data["MSFT"].index.max() + timedelta(days = 1)
    Historical_Data = pickle.load(open(Project_Folder + "Data//Historical_Data.p" , "rb" ))
else:
    start_date = datetime.now() - timedelta(days=N_DAYS_AGO)
    Historical_Data = defaultdict(pandas.DataFrame)

## Pulling Historical Data
for i in tqdm(Assets):
    try:
        Historical_Data[i] =  Historical_Data[i].append(api.polygon.historic_agg_v2(i, 1, 'day', _from=start_date.strftime("%Y-%m-%d"), to=datetime.now().strftime("%Y-%m-%d")).df, ignore_index=True)
    except:
        print(i + " : Failed To Pull Data")

## Saving Raw Data
pickle.dump(Historical_Data, open(Project_Folder + "Data//Historical_Data.p" , "wb" ) )

BMY.RTFailed To Pull Data



In [25]:
# Loading Historical Data If Not Present
if 'Historical_Data' not in locals():
    Historical_Data = pickle.load(open(Project_Folder + "Data//Historical_Data.p" , "rb" ))

## Testing Technical Indicator Calculation
TMP = Historical_Data['MSFT']
ATR = ta.volatility.AverageTrueRange(high = TMP['high'],low = TMP['low'],close = TMP['close'],n = 20,fillna = False).average_true_range()
RSI = ta.momentum.RSIIndicator(close = TMP['close'],n = 20,fillna = False).rsi()
MACD = ta.trend.MACD(close = TMP['close']).macd_diff()